In [1]:
from autometrics.dataset.datasets import RealHumanEval
from autometrics.util.analysis import display_top_5_metrics_by_validation, get_top_metric_by_validation, plot_metric_target_scatterplot
from autometrics.aggregator.regression import Ridge
from autometrics.generator.LLMJudgeProposer import LLMJudgeProposer
import dspy
import litellm
from tqdm import tqdm

litellm.suppress_debug_info = True

/users/mryan0/.conda/envs/autometrics/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
humaneval_prompt = """You are an expert Python programmer, be helpful to the user and return code only in Python."""

task_prompt = humaneval_prompt

# %%
dataset = RealHumanEval()

In [3]:
# %%
train, dev, test = dataset.get_splits(train_ratio=0.2, val_ratio=0.2, seed=42, max_size=1000)

print("Train size:", len(train.dataframe), "Validation size:", len(dev.dataframe), "Test size:", len(test.dataframe))

Train size: 1000 Validation size: 1000 Test size: 1000


In [10]:
train.dataframe['input'].to_list()[:5]

['def count_nums(arr):\n    total = 0\n    total_greater_than_0 = 0\n    for number in arr:\n        string = str(number)\n        print(string)\n        if string[0] == "-":\n            number = -number\n            total += -int(string[1])\n            rest = str(number)[1:]\n            # print(f"the rest: {rest}")\n        else:\n            # print(int(string[0]))\n\n            total = total + int(string[0])\n            # print(f"total: {total}")\n\n            rest = str(number)[1:]\n        print(rest)\n        for dig in rest:\n            total += int(dig)\n            # print(f"dig {total}")\n        if total > 0:\n            total_greater_than_0 +=1\n    return total_greater_than_0\n    \n# print(count_nums([1,1,2]))\nprint(count_nums([1,1,2,-2,3,4,[CODE GOES HERE]2]))\n        \n            \n                \n            \n\n            ',
 'def count_nums(arr):\n    count = 0\n    for num in arr:\n        num_str = str(num)\n        running_sum = 0[CODE GOES HERE]\n  

In [4]:
# %%
llama33_70b = dspy.LM("litellm_proxy/meta-llama/Meta-Llama-3.3-70b-Instruct", api_base="http://future-hgx-1.stanford.edu:7410/v1", api_key="None")

dspy.settings.configure(lm=llama33_70b)

In [5]:
from autometrics.metrics.MetricBank import reference_free_metrics

# %%
generator = LLMJudgeProposer(train_dataset=train, task_description=task_prompt, proposer_model=llama33_70b, judge_model=llama33_70b)

# %%
print(llama33_70b.model)

for target_column in dataset.target_columns:
    train.add_metrics(reference_free_metrics)
    dev.add_metrics(reference_free_metrics)
    test.add_metrics(reference_free_metrics)

df = display_top_5_metrics_by_validation(dev, test, True)
print(df)
df.to_csv("outputs/" + dataset.name + "_top_metrics.csv")

/projects/m000076/mryan0/autometrics/autometrics/metrics/reference_based/MOVERScore.py:25: SyntaxWarning: invalid escape sequence '\m'
  """---


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# %%
new_metrics = reference_free_metrics

for target_column in dataset.target_columns:
    new_metrics.extend(generator.generate(train, target_column))

In [ ]:
# %%
# Condense the metrics that have duplicate names
new_metrics_names = set()
new_metrics_final = []
for metric in new_metrics:
    if metric.name not in new_metrics_names:
        new_metrics_names.add(metric.name)
        new_metrics_final.append(metric)

# %%
train.add_metrics(new_metrics_final)
dev.add_metrics(new_metrics_final)
test.add_metrics(new_metrics_final)

# %%
df = display_top_5_metrics_by_validation(dev, test, True)
print(df)
df.to_csv("outputs/" + dataset.name + "_top_metrics_dspy.csv")

# %%
for target in tqdm(dataset.get_target_columns()):
    aggregator = Ridge(dataset=train, name=f'Ridge_{target}_llm')
    aggregator.ensure_dependencies(train)
    aggregator.ensure_dependencies(dev)
    aggregator.ensure_dependencies(test)

# %%
train.get_metric_columns()

# %%
for target in tqdm(dataset.get_target_columns()):
    aggregator = Ridge(dataset=train, name=f'Ridge_{target}_llm')
    aggregator.learn(train, target)
    aggregator.predict(train)
    aggregator.predict(dev)
    aggregator.predict(test)

# %%
df = display_top_5_metrics_by_validation(dev, test, True)
print(df)
df.to_csv("outputs/" + dataset.name + "_top_metrics_dspy_regression.csv")

# all computed values
dataset.get_dataframe().to_csv("outputs/" + dataset.name + "_all.csv")